In [133]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline


In [134]:
# Load the uploaded CSV files
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

# Display the first few rows of each dataset to understand their structure
train_data.head(), test_data.head()

# Varibale to store the path to the data to make it easier to change


(   id          brand              model  model_year  milage      fuel_type  \
 0   0           MINI      Cooper S Base        2007  213000       Gasoline   
 1   1        Lincoln              LS V8        2002  143250       Gasoline   
 2   2      Chevrolet  Silverado 2500 LT        2002  136731  E85 Flex Fuel   
 3   3        Genesis   G90 5.0 Ultimate        2017   19500       Gasoline   
 4   4  Mercedes-Benz        Metris Base        2021    7388       Gasoline   
 
                                               engine  \
 0       172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel   
 1       252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel   
 2  320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...   
 3       420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel   
 4       208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel   
 
                      transmission ext_col int_col  \
 0                             A/T  Yellow    Gray   
 1                             A/T  Silver   Beige   
 2                

In [135]:
# Assume 'price' is the target variable and others are features
# For simplicity, let's remove any non-numeric columns for now
X = train_data.drop(columns=['price', 'id'])
y = train_data['price']

# Check the data types and head of the dataset to ensure it is correctly processed
X.dtypes, X.head()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [136]:
# Functions to preprocess the data

def exstract_horsepower(df):
    # match = re.search(r'(\d+\.?\d*)HP', str(df))
    # # hp = re.search(r'(\d+)\s*[hH][pP]', df)
    # return float(match.group(1)) if match else None
    if pd.isnull(df):
        return np.nan
    # Ensure the input is a string
    df = str(df)
    # Improving the regex to capture different formats and case variations for 'hp' and 'HP'
    hp = re.search(r'(\d+\.?\d*)HP', df)
    # Extract and return the horsepower as a float
    hp = float(hp.group(1)) if hp else np.nan

    return hp
# Apply the refined horsepower extraction function to the 'engine' column
train_data['engine'] = train_data['engine'].apply(exstract_horsepower)

# Fill the missing values in the 'hp' column with the mean value of the column
mean_hp = train_data['engine'].mean()
train_data['engine'] = train_data['engine'].fillna(mean_hp)

# Function to check if a car has been in an accident
def check_accident(accident_str):
    # If 'accident_str' indicates an accident or damage, return 1, otherwise return 0
    if pd.isnull(accident_str):
        return 0
    accident_str = str(accident_str).lower()
    if "accident" in accident_str or "damage" in accident_str:
        return 1
    return 0

# Apply the function to create a new 'accident_flag' column
train_data['accident'] = train_data['accident'].apply(check_accident)

# Drop the original 'accident' column since we have created the flag
# train_data = train_data.drop(columns=['accident'])

# Define bins and labels for mileage
milage_bins = [0, 50000, 150000, float('inf')]
milage_labels = ['Low', 'Medium', 'High']
train_data['milage_category'] = pd.cut(train_data['milage'], bins=milage_bins, labels=milage_labels)

# Define bins and labels for horsepower
hp_bins = [0, 150, 300, float('inf')]
hp_labels = ['Low', 'Medium', 'High']
train_data['hp_category'] = pd.cut(train_data['engine'], bins=hp_bins, labels=hp_labels)

train_data = train_data.drop(columns=['milage', 'engine'])

# Simplify transmission to 'Manual' or 'Automatic'
def simplify_transmission(trans_str):
    if pd.isnull(trans_str):
        return 'Automatic'  # Assume Automatic if transmission info is missing
    trans_str = trans_str.lower()
    if 'manual' in trans_str or 'm/t' in trans_str:
        return 'Manual'
    else:
        return 'Automatic'

train_data['transmission_type'] = train_data['transmission'].apply(simplify_transmission)
train_data = train_data.drop(columns=['transmission', 'id'])

#One-hot encode the categorical columns
X = pd.get_dummies(train_data, columns=['brand', 'model', 'fuel_type', 'ext_col', 'int_col', 'transmission_type', 'milage_category', 'hp_category', 'transmission_type'])
# Fit the model



train_data.head()

,brand,model,model_year,fuel_type,ext_col,int_col,accident,clean_title,price,milage_category,hp_category,transmission_type
0,MINI,Cooper S Base,2007,Gasoline,Yellow,Gray,0,Yes,4200,High,Medium,Automatic
1,Lincoln,LS V8,2002,Gasoline,Silver,Beige,1,Yes,4999,Medium,Medium,Automatic
2,Chevrolet,Silverado 2500 LT,2002,E85 Flex Fuel,Blue,Gray,0,Yes,13900,Medium,High,Automatic
3,Genesis,G90 5.0 Ultimate,2017,Gasoline,Black,Black,0,Yes,45000,Low,High,Automatic
4,Mercedes-Benz,Metris Base,2021,Gasoline,Black,Beige,0,Yes,97500,Low,Medium,Automatic


In [137]:
# Thresholds for filtering the data


In [138]:
# Update the processing for categorical and numerical features
categorical_features = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'engine', 'accident']
numerical_features = ['model_year', 'milage']

# print(f"Categorical features: {categorical_features}")
# print(f"Numerical features: {numerical_features}")

# Preprocessing for categorical and numerical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Create a pipeline for the Gradient Boosting Regressor
gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor(n_estimators=100, learning_rate=0.3, max_depth=3, random_state=42))
])

# Fit the pipeline
gb_pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred_gb = gb_pipeline.predict(X_val)
mse_gb = mean_squared_error(y_val, y_pred_gb)
print(f"Mean Squared Error: {mse_gb}")

Mean Squared Error: 5167211115.706115
